I got a accuracy of 87 in XGBoost

In [41]:
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [42]:
data = pd.read_csv('new_combined_data.csv')

Distil Bert Model

In [43]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [45]:
data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description', 'num_words_in_transcript',
       'resume_jd_similarity', 'resume_transcript_similarity', 'sentiment',
       'polarity', 'lexical_diversity', 'transcript_length',
       'technical_skill_match', 'soft_skills_sentiment', 'resume_length',
       'job_description_experience_match', 'cultural_fit_sentiment',
       'job_fit_score', 'confidence_score', 'job_desc_complexity',
       'interaction_quality', 'clarity_score', 'text_complexity_transcript',
       'text_complexity_resume'],
      dtype='object')

In [46]:
# Custom Dataset for Batch Processing
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Function to get embeddings batch-wise
def generate_embeddings(texts, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    embeddings = []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            encoded_inputs = tokenizer(
                list(batch), return_tensors='pt', truncation=True, padding=True, max_length=512
            )
            encoded_inputs = {key: val.to(device) for key, val in encoded_inputs.items()}
            outputs = model(**encoded_inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(batch_embeddings)

    return np.vstack(embeddings)


In [47]:
text_features = ['Transcript','Resume','Reason for decision', 'Job Description', 'polarity']
numerical_features = [
    'num_words_in_transcript', 'resume_jd_similarity', 
    'resume_transcript_similarity', 'sentiment',
    'lexical_diversity', 'transcript_length', 'technical_skill_match',
    'soft_skills_sentiment', 'resume_length',
    'job_description_experience_match', 'cultural_fit_sentiment',
    'job_fit_score', 'confidence_score', 'job_desc_complexity',
    'interaction_quality', 'clarity_score', 
    'text_complexity_transcript', 'text_complexity_resume'
]

In [48]:
# Generate embeddings
for feature in text_features:
    print(f"Generating embeddings for {feature}...")
    data[f'{feature}_embedding'] = list(generate_embeddings(data[feature].tolist()))

print("All embeddings generated successfully.")

# Normalize numerical features
scaler = MinMaxScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Concatenate all features
embeddings = np.concatenate(
    [np.vstack(data[f'{feat}_embedding'].to_numpy()) for feat in text_features] +
    [data[numerical_features].to_numpy()], axis=1
)

Generating embeddings for Transcript...
Generating embeddings for Resume...
Generating embeddings for Reason for decision...
Generating embeddings for Job Description...
Generating embeddings for polarity...
All embeddings generated successfully.


In [49]:
# Save concatenated embeddings and features back to the dataset
embedding_df = pd.DataFrame(
    embeddings, 
    columns=[f"feature_{i}" for i in range(embeddings.shape[1])]
)

# Add the target column back for supervised learning
embedding_df['decision'] = data['decision'].values

In [50]:
embedding_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_3849,feature_3850,feature_3851,feature_3852,feature_3853,feature_3854,feature_3855,feature_3856,feature_3857,decision
0,-0.048704,0.125795,0.037339,-0.021360,0.240732,-0.128751,0.042727,0.524011,-0.203181,-0.217218,...,0.833333,0.333333,0.076585,0.461538,0.504845,0.613356,0.772682,0.435219,0.370576,0
1,-0.131237,0.228924,0.118105,-0.036670,0.255953,-0.108508,0.092743,0.456000,-0.134721,-0.204909,...,0.391667,0.333333,0.210348,0.076923,0.779988,0.768511,0.299537,0.401584,0.429927,1
2,-0.098120,0.131073,0.084949,0.080129,0.290251,-0.052578,0.067133,0.533473,-0.178801,-0.157337,...,0.858333,0.333333,0.072764,0.230769,0.703692,0.582768,0.752163,0.312548,0.469971,0
3,-0.127132,0.171015,0.079485,-0.042877,0.263683,-0.154168,0.084967,0.399158,-0.149684,-0.249491,...,0.750000,0.540741,0.164754,0.000000,0.372348,0.688049,0.543351,0.414987,0.483559,1
4,-0.087818,0.120723,0.111387,0.001174,0.273413,-0.131008,0.075665,0.524928,-0.154856,-0.189027,...,0.725000,0.600000,0.060762,0.307692,0.449969,0.632492,0.533092,0.440336,0.607910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,-0.106005,0.161844,0.112026,0.006522,0.264127,-0.141059,-0.004428,0.416072,-0.134533,-0.122462,...,0.133333,0.333333,0.516057,0.153846,0.770451,0.729232,0.555623,0.399964,0.541649,1
3170,-0.123497,0.169966,0.065711,-0.048899,0.324951,0.001595,0.107780,0.273703,-0.015794,-0.267800,...,0.150000,0.333333,0.591477,0.307692,0.762699,0.699965,0.303762,0.313024,0.534925,0
3171,-0.145148,0.067832,0.154981,-0.001577,0.298735,-0.025296,0.049373,0.360152,-0.024444,-0.309433,...,0.166667,0.500000,0.552929,0.000000,0.760098,0.803279,0.533092,0.351979,0.506818,1
3172,-0.140830,0.050024,0.102557,0.015867,0.274238,-0.065866,-0.025638,0.395195,-0.086869,-0.185233,...,0.141667,0.333333,0.556526,0.076923,0.767901,0.740856,0.352645,0.345215,0.402437,0


In [51]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna

# Encode target variable
y = data['decision']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)


In [52]:
# Suppress Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define the objective function
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    # Silent training with verbose=0
    model = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', verbosity=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)

    # Print ROC and AUC score for the current trial
    print(f"Trial completed - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")
    return roc_auc  # Optimize for AUC

# Run the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the best parameters and AUC
print("\nBest parameters:", study.best_params)
print("Best ROC-AUC score:", study.best_value)


Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8790
Trial completed - Accuracy: 0.8882, ROC-AUC: 0.8884
Trial completed - Accuracy: 0.8819, ROC-AUC: 0.8821
Trial completed - Accuracy: 0.8740, ROC-AUC: 0.8742
Trial completed - Accuracy: 0.8819, ROC-AUC: 0.8820
Trial completed - Accuracy: 0.8850, ROC-AUC: 0.8852
Trial completed - Accuracy: 0.8756, ROC-AUC: 0.8758
Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8789
Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8789
Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8788

Best parameters: {'max_depth': 6, 'learning_rate': 0.24995275951355453, 'n_estimators': 413, 'subsample': 0.8759422947199932, 'colsample_bytree': 0.7132565863655246, 'gamma': 4.837741801206061, 'reg_alpha': 2.2534765763206357, 'reg_lambda': 0.6128529451825404}
Best ROC-AUC score: 0.8883724852188406


In [53]:
# Best parameters and final model training
best_params = study.best_params
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'max_depth': 6, 'learning_rate': 0.24995275951355453, 'n_estimators': 413, 'subsample': 0.8759422947199932, 'colsample_bytree': 0.7132565863655246, 'gamma': 4.837741801206061, 'reg_alpha': 2.2534765763206357, 'reg_lambda': 0.6128529451825404}


In [54]:
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
final_model.fit(X_train, y_train)

c:\Users\Durgesh Babu\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [22:19:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7132565863655246, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=4.837741801206061, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.24995275951355453,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=413, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [55]:
# Predictions and evaluation
y_pred = final_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC_AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8881889763779528
ROC_AUC: 0.8883724852188406


In [56]:
y_test_pred_classes = final_model.predict_proba(X_test)
y_test_pred_xgb_distil = np.argmax(y_test_pred_classes, axis=1)

I got a accuracy of 88.8 in distil-bert model

Sentence Transformers

In [57]:
from sentence_transformers import SentenceTransformer

In [58]:
# Define a custom dataset for efficient DataLoader usage
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Function to generate embeddings using CLS Token Pooling with SBERT
def generate_embeddings(texts, model_name='all-MiniLM-L6-v2', batch_size=32, max_length=512):
    """
    Generates embeddings using Sentence Transformers with CLS token pooling.
    Args:
        texts (list): List of texts to embed.
        model_name (str): Pre-trained SentenceTransformer model.
        batch_size (int): Batch size for embedding generation.
        max_length (int): Maximum token length for each text.
    Returns:
        np.ndarray: Generated embeddings.
    """
    # Load the SentenceTransformer model
    model = SentenceTransformer(model_name)
    model.max_seq_length = max_length  # Adjust max token length

    # Dataset and DataLoader for batch processing
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()

    with torch.no_grad():
        for batch in dataloader:
            # Tokenize and encode with CLS token pooling
            batch_embeddings = model.encode(
                batch, 
                batch_size=batch_size,
                convert_to_tensor=True,
                show_progress_bar=False,
                normalize_embeddings=True  # Ensures cosine similarity compatibility
            )
            embeddings.append(batch_embeddings.cpu().numpy())

    return np.vstack(embeddings)

In [59]:
text_features = ['Transcript','Resume','Reason for decision', 'Job Description', 'polarity']
numerical_features = [
    'num_words_in_transcript', 'resume_jd_similarity', 
    'resume_transcript_similarity', 'sentiment',
    'lexical_diversity', 'transcript_length', 'technical_skill_match',
    'soft_skills_sentiment', 'resume_length',
    'job_description_experience_match', 'cultural_fit_sentiment',
    'job_fit_score', 'confidence_score', 'job_desc_complexity',
    'interaction_quality', 'clarity_score', 
    'text_complexity_transcript', 'text_complexity_resume'
]

In [60]:
# Generate embeddings
for feature in text_features:
    print(f"Generating embeddings for {feature}...")
    data[f'{feature}_embedding'] = list(generate_embeddings(data[feature].tolist()))

print("All embeddings generated successfully.")

# Normalize numerical features
scaler = MinMaxScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Concatenate all features
embeddings1 = np.concatenate(
    [np.vstack(data[f'{feat}_embedding'].to_numpy()) for feat in text_features] +
    [data[numerical_features].to_numpy()], axis=1
)

Generating embeddings for Transcript...
Generating embeddings for Resume...
Generating embeddings for Reason for decision...
Generating embeddings for Job Description...
Generating embeddings for polarity...
All embeddings generated successfully.


In [61]:
# Save concatenated embeddings and features back to the dataset
embedding_df1 = pd.DataFrame(
    embeddings1, 
    columns=[f"feature_{i}" for i in range(embeddings1.shape[1])]
)

# Add the target column back for supervised learning
embedding_df1['decision'] = data['decision'].values

In [62]:
embedding_df1

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1929,feature_1930,feature_1931,feature_1932,feature_1933,feature_1934,feature_1935,feature_1936,feature_1937,decision
0,-0.087589,0.084175,-0.008520,-0.027508,-0.015328,-0.066478,0.022890,0.072765,-0.064546,-0.006970,...,0.833333,0.333333,0.076585,0.461538,0.504845,0.613356,0.772682,0.435219,0.370576,0
1,-0.136713,0.047159,0.006397,0.018619,-0.029415,-0.154497,0.031236,0.062624,-0.099282,-0.014918,...,0.391667,0.333333,0.210348,0.076923,0.779988,0.768511,0.299537,0.401584,0.429927,1
2,-0.054821,0.003179,0.014176,0.047195,-0.010605,-0.149051,0.037159,0.027634,-0.082209,-0.006898,...,0.858333,0.333333,0.072764,0.230769,0.703692,0.582768,0.752163,0.312548,0.469971,0
3,-0.114528,0.045982,0.014623,0.021302,-0.026010,-0.152267,0.000823,0.059678,-0.088712,-0.031706,...,0.750000,0.540741,0.164754,0.000000,0.372348,0.688049,0.543351,0.414987,0.483559,1
4,-0.095079,0.028903,0.009564,-0.004095,-0.039646,-0.174000,0.028329,0.014707,-0.103673,-0.008380,...,0.725000,0.600000,0.060762,0.307692,0.449969,0.632492,0.533092,0.440336,0.607910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,-0.104514,-0.009513,-0.034277,0.048185,-0.053155,-0.088607,0.084419,-0.011694,-0.044390,0.022772,...,0.133333,0.333333,0.516057,0.153846,0.770451,0.729232,0.555623,0.399964,0.541649,1
3170,-0.062923,0.023802,0.034703,-0.014107,0.033172,-0.026891,0.136209,0.053655,-0.021299,-0.007582,...,0.150000,0.333333,0.591477,0.307692,0.762699,0.699965,0.303762,0.313024,0.534925,0
3171,-0.051464,0.021795,0.033847,0.069617,0.051025,-0.052515,-0.002937,0.082780,-0.089010,-0.055619,...,0.166667,0.500000,0.552929,0.000000,0.760098,0.803279,0.533092,0.351979,0.506818,1
3172,-0.106752,0.019520,0.020833,0.008391,-0.059304,-0.083716,0.033461,0.041888,-0.026378,0.050847,...,0.141667,0.333333,0.556526,0.076923,0.767901,0.740856,0.352645,0.345215,0.402437,0


In [63]:
# Encode target variable
y = data['decision']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings1, y, test_size=0.2, random_state=42)

In [64]:
# Suppress Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define the objective function
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    # Silent training with verbose=0
    model = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', verbosity=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)

    # Print ROC and AUC score for the current trial
    print(f"Trial completed - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")
    return roc_auc  # Optimize for AUC

# Run the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the best parameters and AUC
print("\nBest parameters:", study.best_params)
print("Best ROC-AUC score:", study.best_value)


Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8789
Trial completed - Accuracy: 0.8882, ROC-AUC: 0.8884
Trial completed - Accuracy: 0.8724, ROC-AUC: 0.8726
Trial completed - Accuracy: 0.8882, ROC-AUC: 0.8883
Trial completed - Accuracy: 0.8819, ROC-AUC: 0.8820
Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8789
Trial completed - Accuracy: 0.8882, ROC-AUC: 0.8884
Trial completed - Accuracy: 0.8866, ROC-AUC: 0.8867
Trial completed - Accuracy: 0.8835, ROC-AUC: 0.8836
Trial completed - Accuracy: 0.8835, ROC-AUC: 0.8836

Best parameters: {'max_depth': 10, 'learning_rate': 0.040933457124752964, 'n_estimators': 272, 'subsample': 0.7678769535883538, 'colsample_bytree': 0.7254623400865843, 'gamma': 3.7113210929144635, 'reg_alpha': 1.3395919312015736, 'reg_lambda': 7.4031312598076084}
Best ROC-AUC score: 0.8883873655807308


In [65]:
best_params = study.best_params
final_model2 = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
final_model2.fit(X_train, y_train)

c:\Users\Durgesh Babu\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [22:40:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7254623400865843, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=3.7113210929144635, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.040933457124752964,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=272, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [66]:
# Predictions and evaluation
y_pred = final_model2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC_AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8881889763779528
ROC_AUC: 0.8883873655807308


In [67]:
y_test_pred_classes = final_model2.predict_proba(X_test)
y_test_pred_xgb_sen= np.argmax(y_test_pred_classes, axis=1)

In sentence transformer I got a accuracy of 88.81

ANN

In [68]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Function to create and train the model for Optuna optimization
def objective(trial):
    model = Sequential()
    
    # Hyperparameter tuning for number of units and layers
    units_1 = trial.suggest_int('units_1', 32, 256, step=32)
    dropout_1 = trial.suggest_float('dropout_1', 0.1, 0.5, step=0.1)
    units_2 = trial.suggest_int('units_2', 32, 128, step=32)
    dropout_2 = trial.suggest_float('dropout_2', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    # Build the model
    model.add(Dense(units=units_1, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(rate=dropout_1))
    model.add(Dense(units=units_2, activation='relu'))
    model.add(Dropout(rate=dropout_2))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=0)
    
    # Evaluate the model on validation data
    score = model.evaluate(X_test, y_test, verbose=0)
    return score[1]  # Return validation accuracy

# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Get the best parameters and train the final model
best_params = study.best_params
print("Best Hyperparameters:", best_params)



C:\Users\Durgesh Babu\AppData\Local\Temp\ipykernel_12016\1609942850.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


Best Hyperparameters: {'units_1': 32, 'dropout_1': 0.5, 'units_2': 96, 'dropout_2': 0.1, 'learning_rate': 0.00012570089547090147}


In [69]:
# Final model using the best parameters
final_model1 = Sequential()
final_model1.add(Dense(units=best_params['units_1'], activation='relu', input_dim=X_train.shape[1]))
final_model1.add(Dropout(rate=best_params['dropout_1']))
final_model1.add(Dense(units=best_params['units_2'], activation='relu'))
final_model1.add(Dropout(rate=best_params['dropout_2']))
final_model1.add(Dense(1, activation='sigmoid'))

final_model1.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])

# Train the final model
final_model1.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the final model
final_accuracy = final_model1.evaluate(X_test, y_test)
print(f"Final Model Test Accuracy: {final_accuracy[1]:.4f}")


Epoch 1/100
159/159 [==============================] - 1s 4ms/step - loss: 0.6639 - accuracy: 0.6633 - val_loss: 0.6158 - val_accuracy: 0.8504
Epoch 2/100
159/159 [==============================] - 1s 3ms/step - loss: 0.5621 - accuracy: 0.8094 - val_loss: 0.4681 - val_accuracy: 0.8787
Epoch 3/100
159/159 [==============================] - 1s 3ms/step - loss: 0.4293 - accuracy: 0.8570 - val_loss: 0.3267 - val_accuracy: 0.8756
Epoch 4/100
159/159 [==============================] - 1s 3ms/step - loss: 0.3201 - accuracy: 0.8791 - val_loss: 0.2551 - val_accuracy: 0.8772
Epoch 5/100
159/159 [==============================] - 1s 3ms/step - loss: 0.2728 - accuracy: 0.8862 - val_loss: 0.2262 - val_accuracy: 0.8803
Epoch 6/100
159/159 [==============================] - 1s 3ms/step - loss: 0.2404 - accuracy: 0.8937 - val_loss: 0.2002 - val_accuracy: 0.8882
Epoch 7/100
159/159 [==============================] - 1s 3ms/step - loss: 0.2243 - accuracy: 0.8941 - val_loss: 0.1977 - val_accuracy: 0.8819

In ANN, I got a accuracy of 88.66

In [70]:
y_test_pred_nn = final_model1.predict(X_test)

20/20 [==============================] - 0s 2ms/step


In [71]:
test_df = pd.DataFrame()
test_df['actual'] = y_test
test_df['xg_distil_bert'] = y_test_pred_xgb_distil
test_df['xg_sen_transformer'] = y_test_pred_xgb_sen
test_df['nn_prediction'] = y_test_pred_nn

In [72]:
test_df

,actual,xg_distil_bert,xg_sen_transformer,nn_prediction
2270,1,1,1,1.000000e+00
442,1,1,1,1.000000e+00
2885,1,1,1,1.000000e+00
1655,1,0,0,9.431745e-01
1001,0,0,0,3.850716e-11
...,...,...,...,...
282,0,0,0,9.354323e-10
1953,1,1,0,7.738764e-01
691,0,0,0,3.063560e-11
794,0,0,0,3.768873e-12


In [73]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


def calculate_metrics(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_pred),
    }

# Majority voting
test_df['ensemble_vote'] = (test_df[['xg_distil_bert', 'xg_sen_transformer', 'nn_prediction']].mean(axis=1) > 0.5).astype(int)

# Evaluate ensemble performance
ensemble_metrics = calculate_metrics(test_df['actual'], test_df['ensemble_vote'])
print("Ensemble Metrics:", ensemble_metrics)

Ensemble Metrics: {'Accuracy': 0.8913385826771654, 'Precision': 0.9194630872483222, 'Recall': 0.8589341692789969, 'F1-Score': 0.8881685575364667, 'ROC-AUC': 0.8914924010951947}


After ensembling, we got a increased accuracy from 87 to 89.1

In [74]:
# Mean

test_df['mean_prob'] = (test_df['xg_distil_bert'] + test_df['xg_sen_transformer'] + test_df['nn_prediction'])/3

In [81]:
test_df['new_pred'] = test_df['mean_prob'].round()
test_df['new_nn_prediction'] = test_df['nn_prediction'].round()
test_df

,actual,xg_distil_bert,xg_sen_transformer,nn_prediction,ensemble_vote,mean_prob,new_pred,new_nn_prediction
2270,1,1,1,1.000000e+00,1,1.000000e+00,1.0,1.0
442,1,1,1,1.000000e+00,1,1.000000e+00,1.0,1.0
2885,1,1,1,1.000000e+00,1,1.000000e+00,1.0,1.0
1655,1,0,0,9.431745e-01,0,3.143915e-01,0.0,1.0
1001,0,0,0,3.850716e-11,0,1.283572e-11,0.0,0.0
...,...,...,...,...,...,...,...,...
282,0,0,0,9.354323e-10,0,3.118108e-10,0.0,0.0
1953,1,1,0,7.738764e-01,1,5.912921e-01,1.0,1.0
691,0,0,0,3.063560e-11,0,1.021187e-11,0.0,0.0
794,0,0,0,3.768873e-12,0,1.256291e-12,0.0,0.0


Accuracy for actual with ensembing

In [76]:
print(accuracy_score(test_df['actual'], test_df['new_pred']))
print(roc_auc_score(test_df['actual'], test_df['new_pred']))

0.8913385826771654
0.8914924010951947


Accuracy for actual with distil bert

In [77]:
print(accuracy_score(test_df['actual'], test_df['xg_distil_bert']))
print(roc_auc_score(test_df['actual'], test_df['xg_distil_bert']))

0.8881889763779528
0.8883724852188406


Accuracy for actual with sentence transformer

In [78]:
print(accuracy_score(test_df['actual'], test_df['xg_sen_transformer']))
print(roc_auc_score(test_df['actual'], test_df['xg_sen_transformer']))

0.8881889763779528
0.8883873655807308


Accuracy for actual with ANN

In [82]:
print(accuracy_score(test_df['actual'], test_df['new_nn_prediction']))
print(roc_auc_score(test_df['actual'], test_df['new_nn_prediction']))

0.8866141732283465
0.8866414031189238


In [83]:
print(roc_auc_score(test_df['actual'], test_df['nn_prediction']))

0.9763501448355223
